# Veri Analizleri

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
from plotly import express
from plotly import graph_objects
from plotly import io

io.templates.default = "plotly_dark"

In [3]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.tsa.stattools import adfuller, acf, breakvar_heteroskedasticity_test

In [4]:
from statsmodels.graphics import tsaplots
import matplotlib.pyplot as plt

In [5]:
import plotly.graph_objects as go
import statsmodels.api as sm 

In [6]:
country_name = ["France", "Greece",	"Italy", "Portugal", "Spain", "Türkiye"]

In [7]:
df_2 = pd.read_excel("OECD VERİLERİ ELİF BAĞCI TEZ V0_python.xlsx", sheet_name=6, header=1 )
df_2.head()

,Unnamed: 0,France,Greece,Italy,Portugal,Spain,Türkiye,t,interruption,interrupted t
0,2000,21.120,38.374,27.362,30.222,28.642,38.329,1,0,0
1,2001,21.088,36.794,25.377,29.787,28.971,32.749,2,0,0
2,2002,20.825,39.542,25.041,27.964,28.960,30.026,3,0,0
3,2003,21.222,40.136,24.666,29.352,28.896,28.793,4,0,0
4,2004,21.214,41.376,23.756,29.394,28.650,28.297,5,0,0


In [8]:
def create_corr_plot(series, plot_pacf=False):
    corr_array = pacf(series.dropna(), alpha=0.05) if plot_pacf else acf(series.dropna(), alpha=0.05)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]

    fig = go.Figure()
    [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines',line_color='#3f3f3f') 
     for x in range(len(corr_array[0]))]
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                   marker_size=12)
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
            fill='tonexty', line_color='rgba(255,255,255,0)')
    fig.update_traces(showlegend=False)
    fig.update_xaxes(range=[-1,13])
    fig.update_yaxes(zerolinecolor='#000000')
    
    title='Partial Autocorrelation (PACF)' if plot_pacf else 'Autocorrelation (ACF)'
    fig.update_layout(title=title)
    fig.show()

In [9]:
for country in country_name:
    print("###########################")
    print(f"{country}'nin Zaman Serisi Analizi")
    print("###########################")

    trendlin = express.scatter(
        df_2,
        x="t",
        y=f"{country}",
        title=f"{country}'s Voluntary schemes/household out-of-pocket payments/% of current expenditure on health",
        trendline = "ols"
    )

    trendlin.show()
    linear_model = ols(f"{country} ~ t", data=df_2).fit()
    print(linear_model.summary())
    print(" ", end="\n")
    print("###########################")
    print("Verinin Durağanlık Kontrolü", end="\n\n")
    print("H0: Seri durağan değildir.")
    print("H1: Seri durağandır.", end="\n\n")
    adfullerx = adfuller(df_2[[f"{country}"]])
    print(f'Test Stat = {round(adfullerx[0], 4)}, p-value = {round(adfullerx[1], 4)}')

    if adfullerx[1]<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Yani seri durağandır.")
    elif adfullerx[1]>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani seri durağan değildir.", end="\n\n")
    print("###########################")
    print("Hataların Normal Dağılması", end="\n\n")
    residual_values = linear_model.resid

    print("H0: Normal dağılım varsayımı sağlanmaktadır.")
    print("H1: Normal dağılım varsayımı sağlanmamaktadır.", end="\n\n")

    test_stat, pvalue = stats.shapiro(residual_values)
    print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue, 4)}')
    if pvalue<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar normal dağılmamıştır.")
    elif pvalue>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar normal dağılmıştır.", end="\n\n")
    hist = express.histogram(residual_values, title="Hataların Dağılımı")
    hist.show()

    print("###########################")
    print("Hataların Rassallığı", end="\n\n")

    print("     Hataların Durağanlığı", end="\n\n")

    adfullery = adfuller(residual_values)
    print(f'Test Stat = {round(adfullery[0], 4)}, p-value = {round(adfullery[1], 4)}')

    if adfullery[1]<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar durağandır.")
    elif adfullery[1]>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.", end="\n\n")

    resid_scatter = express.scatter(
        df_2,
        x="t",
        y=residual_values,
        title="Hataların Rassallığı"
    )

    resid_scatter.show()


    print("     Hataların Otokorelasyonu", end="\n\n")

    print("Ljung-Box q istatistiğinin Hipotezleri")
    print("H0: Veriler bağımsız (random) olarak dağılmıştır.")
    print("H1: Veriler bağımsız (random) olarak dağılmamıştır.", end="\n\n")

    acf_df = pd.DataFrame(pd.DataFrame(acf(residual_values, qstat=True)).T)
    acf_df.columns = ["ACF", "Ljung-Box q statistic", "p-value"]
    display(acf_df[1:6])

    create_corr_plot(residual_values)


###########################
France'nin Zaman Serisi Analizi
###########################


                            OLS Regression Results                            
Dep. Variable:                 France   R-squared:                       0.440
Model:                            OLS   Adj. R-squared:                  0.411
Method:                 Least Squares   F-statistic:                     14.95
Date:                Tue, 31 Jan 2023   Prob (F-statistic):            0.00104
Time:                        19:08:22   Log-Likelihood:                -45.662
No. Observations:                  21   AIC:                             95.32
Df Residuals:                      19   BIC:                             97.41
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     23.7648      1.013     23.469      0.0

###########################
Hataların Rassallığı

     Hataların Durağanlığı

Test Stat = -1.5087, p-value = 0.5292
p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.



     Hataların Otokorelasyonu

Ljung-Box q istatistiğinin Hipotezleri
H0: Veriler bağımsız (random) olarak dağılmıştır.
H1: Veriler bağımsız (random) olarak dağılmamıştır.



,ACF,Ljung-Box q statistic,p-value
1,0.756300,20.690175,0.000032
2,0.520104,22.751351,0.000046
3,0.277153,22.813850,0.000138
4,0.046902,23.778167,0.000239
5,-0.178730,28.208452,0.000086


...

###########################
Türkiye'nin Zaman Serisi Analizi
###########################


                            OLS Regression Results                            
Dep. Variable:                Türkiye   R-squared:                       0.670
Model:                            OLS   Adj. R-squared:                  0.652
Method:                 Least Squares   F-statistic:                     38.54
Date:                Tue, 31 Jan 2023   Prob (F-statistic):           5.78e-06
Time:                        19:08:22   Log-Likelihood:                -52.665
No. Observations:                  21   AIC:                             109.3
Df Residuals:                      19   BIC:                             111.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     33.3618      1.413     23.604      0.0

###########################
Hataların Rassallığı

     Hataların Durağanlığı

Test Stat = -2.6774, p-value = 0.078
p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.



     Hataların Otokorelasyonu

Ljung-Box q istatistiğinin Hipotezleri
H0: Veriler bağımsız (random) olarak dağılmıştır.
H1: Veriler bağımsız (random) olarak dağılmamıştır.



,ACF,Ljung-Box q statistic,p-value
1,0.545345,7.939807,0.018875
2,0.172629,8.266051,0.040822
3,-0.110264,10.422580,0.033880
4,-0.275504,10.678810,0.058134
5,-0.092130,10.837158,0.093542


# Interrupted Time Series for Türkiye

    Yukarıda referans ülkelerin ve Türkiye'nin zaman serisi regresyon analizi sonuçlarına bakılırken, bu kısımdan itibaren Türkiye için interrupted time series analysis yapılmıştır.

In [10]:
X = df_2[["t", "interruption", "interrupted t"]]
y = df_2["Türkiye"]

X = sm.add_constant(X) 

analys= sm.OLS(y, X).fit()

trendlin = express.scatter(
        df_2,
        x="t",
        y= "Türkiye",
        title="Türkiye's Voluntary schemes/household out-of-pocket payments/% of current expenditure on health",
    )

trendlin.add_trace(
go.Scatter(
    x = [1,5],
    y = [analys.predict()[0], analys.predict()[4]],
)
 )   

trendlin.add_trace(
go.Scatter(
    x = [5,6],
    y = [analys.predict()[4], analys.predict()[5]]
)

 )

trendlin.add_trace(
go.Scatter(
    x = [6,21],
    y = [analys.predict()[5], analys.predict()[20]]
)

 )  

trendlin.show() 



print(analys.summary())

print("###########################")
print("Hataların Normal Dağılması", end="\n\n")
residual_values = analys.resid
print("H0: Normal dağılım varsayımı sağlanmaktadır.")
print("H1: Normal dağılım varsayımı sağlanmamaktadır.", end="\n\n")
test_stat, pvalue = stats.shapiro(residual_values)
print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue, 4)}')
if pvalue<0.05:
    print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar normal dağılmamıştır.")
elif pvalue>0.05:
     print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar normal dağılmıştır.", end="\n\n")
hist = express.histogram(residual_values, title="Hataların Dağılımı")
hist.show()
print("###########################")
print("Hataların Rassallığı", end="\n\n")
print("     Hataların Durağanlığı", end="\n\n")
adfullery = adfuller(residual_values)
print(f'Test Stat = {round(adfullery[0], 4)}, p-value = {round(adfullery[1], 4)}')
if adfullery[1]<0.05:
    print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar durağandır.")
elif adfullery[1]>0.05:
    print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.", end="\n\n")
resid_scatter = express.scatter(
    df_2,
    x="t",
    y=residual_values,
    title="Hataların Rassallığı"
)
resid_scatter.show()
print("     Hataların Otokorelasyonu", end="\n\n")
print("Ljung-Box q istatistiğinin Hipotezleri")
print("H0: Veriler bağımsız (random) olarak dağılmıştır.")
print("H1: Veriler bağımsız (random) olarak dağılmamıştır.", end="\n\n")
acf_df = pd.DataFrame(pd.DataFrame(acf(residual_values, qstat=True)).T)
acf_df.columns = ["ACF", "Ljung-Box q statistic", "p-value"]
display(acf_df[1:6])
create_corr_plot(residual_values)

                            OLS Regression Results                            
Dep. Variable:                Türkiye   R-squared:                       0.730
Model:                            OLS   Adj. R-squared:                  0.683
Method:                 Least Squares   F-statistic:                     15.33
Date:                Tue, 31 Jan 2023   Prob (F-statistic):           4.37e-05
Time:                        19:08:22   Log-Likelihood:                -50.546
No. Observations:                  21   AIC:                             109.1
Df Residuals:                      17   BIC:                             113.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            38.8448      3.131     12.407

###########################
Hataların Rassallığı

     Hataların Durağanlığı

Test Stat = -3.6019, p-value = 0.0057
p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar durağandır.


     Hataların Otokorelasyonu

Ljung-Box q istatistiğinin Hipotezleri
H0: Veriler bağımsız (random) olarak dağılmıştır.
H1: Veriler bağımsız (random) olarak dağılmamıştır.



,ACF,Ljung-Box q statistic,p-value
1,0.639759,11.593548,0.003037
2,0.259295,11.699125,0.008488
3,-0.062726,15.175824,0.004350
4,-0.349812,19.771933,0.001379
5,-0.390195,23.269251,0.000711
